6.6.2019

### Image Processing in Physics
#### Julia Herzen, Klaus Achterhold, Fabio De Marco, Manuel Schultheiß

# Exercise 6, Task 2: Connected Components
Have you ever woundered how a battery looks inside?
This exercise will answer all your urgent quesions!

As batteries are produced on a large scale nowadays, non-destructive testing to maintain battery safety can be performed by computed tomography, for example (Further information not needed to solve the exercise: https://www.nature.com/articles/ncomms7924).
We performed a CT scan of a 9V block battery for you and your task is to segment the battery cells using a connected component algorithm and thresholding. Afterwards you determine the median and mean intensity for each battery cell and plot them.

Please note you need to install scikit image to solve this exercise (https://scikit-image.org/docs/dev/install.html).

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive
from skimage.measure import label, regionprops

# Load a CT scan of a battery
battery = np.load("battery.npy")[:, ::2, ::2]

**Task 1: Min-Max Normlization: **
First we want to normalize the intensity values to a $[0, 1]$ range (e.g. the highest value in the array should be 1 and the lowest value should be 0).

In the original CT scan, the intesities may also have negative values such as -2. Use `battery.min()` and `battery.max()` to find the minimum and maximum.  Mathematically, from each voxel $v_i$ the minimum instensity of the whole scan is subtracted and afterwards it is divided by the intensity range:


$$\mathrm{v}_i=\frac{\mathrm{v}_i-\min(\mathrm{battery})}{\max(\mathrm{battery})-\min(\mathrm{battery})}$$

In [ ]:
battery = ???

Some assertion code to ensure everything was implemented correctely:

In [ ]:
assert battery.max()==1 and battery.min()==0

The following function displays a 3D scan for you, where you can inspect the slice stack by using a slider.

In [ ]:
def show_ct(ctscan, colors=False):
    def f( ct_slice_index):
        fig, ax=plt.subplots(dpi=200)
        ax.imshow(ctscan[ct_slice_index], cmap="gray" if not colors else "viridis",   vmin=0, vmax=1)

    interactive_plot = interactive(f, ct_slice_index=(0,9))
    output = interactive_plot.children[-1]
    display(interactive_plot)
           
show_ct(battery)

**Task 2: Binary Thresholding** Your task is to threshold the scan to a value above 0.42. `thresholded_battery` battery should contain `True` for values > 0.42 and `False` for other voxels.

    

In [ ]:
thresholded_battery = ???
show_ct(thresholded_battery.astype(np.int32))

**Task 3: Connected components ** Use the label function from skimage to assign an unique integer value to each connected group of voxels

In [ ]:
label_image =  ???

We can inspect the result using our `plt.imshow` function for the 4th slice:

In [ ]:
plt.imshow(label_image[3])

**Task 4: Extract Battery Cells: **  Battery cells in our scan have between 4000 and 6000 voxels. Add the `region.bbox` property of regions with a voxel count within that range to the list `regions`. You can access the voxel count of each connected component using `region.area`.

In [ ]:
regions = []

for region in regionprops(label_image):
    if region.area >= ??? and region.area < ???: # ???
        regions.append(region.bbox)

Next we want to show each battery cell. This helper function will return a subvolume when providing a battery cell number.

In [ ]:
def get_cell(cell_index):
    """
        Args: 
            cell_index: The cell number. Can be 1,2,3,4,5 or 6
    """
    
    start_dim0 = regions[cell_index][0]
    end_dim0 =  regions[cell_index][3]
    
    start_dim1 = regions[cell_index][1]
    end_dim1 =  regions[cell_index][4]
    
    start_dim2 = regions[cell_index][2]
    end_dim2 =  regions[cell_index][5]
    
    return battery[start_dim0:end_dim0,start_dim1:end_dim1,start_dim2:end_dim2]

In [ ]:
# Show the 3D volume of cell 1
show_ct(get_cell(1))

**Task 5: Plot Median and Mean **  
Next, we want to extract mean intensity and maximum intensity for each cell and plot it into a scatterplot. Hereby, we create a colormap first. Your task is to extract the mean and median intensity from each slice in each cell (Consequently you need to have 48 values for mean and median each, as there are 6 cells with 8 slices each). Plot these values using a scatterplot, wherby the x-axis defines the mean instensiy and the y-axis defines the median intensity.

In [ ]:
import matplotlib.cm as cm
colormap = cm.rainbow(np.linspace(0, 1, 6)) 

means = []
medians = []
colors = []

for cell in range(0,6):
    for slice_index in range(1,9): # We do not use the first and the last slice
        means.append(???) 
        medians.append(???)
        colors.append(colormap[cell])


plt.title("Battery Features")
plt.xlabel("Mean Intensity")
plt.ylabel("Median Intensity")

plt.scatter(means, medians, color=colors)
plt.show()